In [26]:


import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
     

df = pd.read_csv('/Users/parasnath/Desktop/GenerativeAI_ML/FastApi/insurance.csv')
     

df.sample(5)
     



age  weight  height  income_lpa  smoker        city      occupation  \
22   57   106.4    1.83   30.000000   False  Chandigarh  government_job   
69   46    67.2    1.75    6.034487   False     Lucknow  government_job   
63   47    71.3    1.82   41.660000    True        Gaya  business_owner   
81   41    82.6    1.61   22.190000    True      Mysore      freelancer   
2    39    56.8    1.64   36.640000   False      Indore      freelancer   

   insurance_premium_category  
22                        Low  
69                        Low  
63                       High  
81                       High  
2                         Low

In [28]:
df_feat = df.copy()

In [30]:
# Feature 2: Age Group
def age_group(age):
    if age < 25:
        return "young"
    elif age < 45:
        return "adult"
    elif age < 60:
        return "middle_aged"
    return "senior"

In [31]:
df_feat["age_group"] = df_feat["age"].apply(age_group)

In [32]:
# Feature 3: Lifestyle Risk
def lifestyle_risk(row):
    if row["smoker"] and row["bmi"] > 30:
        return "high"
    elif row["smoker"] or row["bmi"] > 27:
        return "medium"
    else:
        return "low"

In [33]:
df_feat["lifestyle_risk"] = df_feat.apply(lifestyle_risk, axis=1)

In [34]:
tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"
]


In [35]:
# Feature 4: City Tier
def city_tier(city):
    if city in tier_1_cities:
        return 1
    elif city in tier_2_cities:
        return 2
    else:
        return 3

In [36]:
df_feat["city_tier"] = df_feat["city"].apply(city_tier)
     

In [37]:
df_feat.drop(columns=['age', 'weight', 'height', 'smoker', 'city'])[['income_lpa', 'occupation', 'bmi', 'age_group', 'lifestyle_risk', 'city_tier', 'insurance_premium_category']].sample(5)

income_lpa   occupation        bmi    age_group lifestyle_risk  city_tier  \
59        1.13      retired  35.835044       senior         medium          2   
23       23.71   unemployed  22.187855        adult            low          2   
34        0.68      retired  32.914286       senior           high          3   
24       18.60  private_job  19.669038  middle_aged            low          2   
3         3.34      student  45.535900        young           high          1   

   insurance_premium_category  
59                       High  
23                     Medium  
34                       High  
24                     Medium  
3                      Medium

In [39]:


# Select features and target
X = df_feat[["bmi", "age_group", "lifestyle_risk", "city_tier", "income_lpa", "occupation"]]
y = df_feat["insurance_premium_category"]
     


In [40]:
# Define categorical and numeric features
categorical_features = ["age_group", "lifestyle_risk", "occupation", "city_tier"]
numeric_features = ["bmi", "income_lpa"]
     

In [41]:
# Create column transformer for OHE
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), categorical_features),
        ("num", "passthrough", numeric_features)
    ]
)
     

In [42]:
# Create a pipeline with preprocessing and random forest classifier
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])
     

In [43]:
# Split data and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['age_group',
                                                   'lifestyle_risk',
                                                   'occupation', 'city_tier']),
                                                 ('num', 'passthrough',
                                                  ['bmi', 'income_lpa'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [44]:
# Predict and evaluate
y_pred = pipeline.predict(X_test)
accuracy_score(y_test, y_pred)

0.9

In [45]:
X_test.sample(5)

bmi    age_group lifestyle_risk  city_tier  income_lpa  \
32  31.495845  middle_aged         medium          2       50.00   
51  38.827923  middle_aged           high          2       28.95   
93  23.199416        young            low          2        1.28   
92  18.319942        adult         medium          2       30.00   
10  22.949982        adult         medium          1       32.78   

        occupation  
32     private_job  
51     private_job  
93         student  
92  government_job  
10  business_owner

In [46]:
import pickle

# Save the trained pipeline using pickle
pickle_model_path = "model.pkl"
with open(pickle_model_path, "wb") as f:
    pickle.dump(pipeline, f)